In [1]:
# ruff: noqa: F401

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import time
from datetime import datetime
from typing import Any, cast

import hvplot as hv
import hvplot.polars
import numpy as np
import polars as pl

from encomp.fluids import Fluid, Water
from encomp.misc import isinstance_types
from encomp.units import Quantity as Q
from encomp.units import Unit
from encomp.utypes import (
    DT,
    MT,
    Currency,
    Density,
    Dimensionality,
    Dimensionless,
    Length,
    Mass,
    Numpy1DArray,
    Power,
    Pressure,
    Temperature,
    TemperatureDifference,
    Time,
    UnitsContainer,
    UnknownDimensionality,
)

In [4]:
# Properties to compare
attrs = ("D", "H", "S")

N_grid = 100

In [5]:
def get_ratio_stats(df: pl.DataFrame, attrs: tuple[str, ...]) -> dict[str, dict]:
    stats = {}
    for a in attrs:
        ratio = (df[f"w95_{a}"] / df[f"w97_{a}"]).to_numpy()
        # Filter out any inf/nan values
        ratio = ratio[np.isfinite(ratio)]
        if len(ratio) > 0:
            stats[a] = {
                "min": np.min(ratio),
                "max": np.max(ratio),
                "mean": np.mean(ratio),
                "std": np.std(ratio),
            }
    return stats

In [6]:
# ===== Heatmap 1: Temperature vs Pressure =====
print("Calculating T vs P grid...")

# Create 2D grid
T_range = np.linspace(25, 700, N_grid)  # 25 to 700 degC
P_range = np.linspace(0.1, 200, N_grid)  # 0.1 to 200 bar
T_grid, P_grid = np.meshgrid(T_range, P_range)

# Flatten for calculation
T_flat = cast(Q[Temperature, Numpy1DArray], Q(T_grid.flatten(), "degC"))
P_flat = cast(Q[Pressure, Numpy1DArray], Q(P_grid.flatten(), "bar"))

# Calculate properties for both formulations
w97_tp = Fluid("IF97::Water", T=T_flat, P=P_flat)
w95_tp = Fluid("Water", T=T_flat, P=P_flat)

# Create dataframe
df_tp = pl.DataFrame(
    {
        "T": T_flat.m,
        "P": P_flat.m,
    }
    | {f"w95_{a}": getattr(w95_tp, a).m for a in attrs}
    | {f"w97_{a}": getattr(w97_tp, a).m for a in attrs}
)

# Get statistics for dynamic color scaling
stats_tp = get_ratio_stats(df_tp, attrs)
print("\nRatio statistics (IAPWS-95 / IAPWS-97) for T vs P:")
for a in attrs:
    if a in stats_tp:
        s = stats_tp[a]
        pct_diff = (s["mean"] - 1.0) * 100
        pct_range = (s["max"] - s["min"]) * 100
        print(
            f"  {a}: {s['min']:.6f} - {s['max']:.6f} (mean: {s['mean']:.6f}, "
            f"Δ={pct_diff:+.3f}%, range={pct_range:.3f}%)"
        )

# Create heatmaps with dynamic color limits
plots_tp = []
for a in attrs:
    if a not in stats_tp:
        continue

    # Calculate percentage difference instead of ratio for better visualization
    df_plot = df_tp.with_columns(((pl.col(f"w95_{a}") / pl.col(f"w97_{a}") - 1.0) * 100).alias(f"pct_diff_{a}"))

    # Use symmetric color limits around 0
    s = stats_tp[a]
    max_abs_pct = max(abs((s["min"] - 1.0) * 100), abs((s["max"] - 1.0) * 100))
    clim = (-max_abs_pct, max_abs_pct) if max_abs_pct > 0.001 else (-0.1, 0.1)

    plot = df_plot.hvplot.heatmap(
        x="T",
        y="P",
        C=f"pct_diff_{a}",
        title=f"{a}: % Difference (95-97)/97 (T vs P)",
        xlabel="Temperature (°C)",
        ylabel="Pressure (bar)",
        cmap="RdBu_r",
        clim=clim,
        colorbar=True,
        width=500,
        height=400,
        clabel="% Difference",
    )
    plots_tp.append(plot)

print("\nT vs P plots ready")
sum(plots_tp[1:], plots_tp[0]).cols(2)

Calculating T vs P grid...

Ratio statistics (IAPWS-95 / IAPWS-97) for T vs P:
  D: 0.999086 - 1.000322 (mean: 1.000003, Δ=+0.000%, range=0.124%)
  H: 0.999823 - 1.000327 (mean: 1.000006, Δ=+0.001%, range=0.050%)
  S: 0.999858 - 1.000259 (mean: 1.000012, Δ=+0.001%, range=0.040%)

T vs P plots ready


:Layout
   .HeatMap.I   :HeatMap   [T,P]   (pct_diff_D)
   .HeatMap.II  :HeatMap   [T,P]   (pct_diff_H)
   .HeatMap.III :HeatMap   [T,P]   (pct_diff_S)

In [7]:
# ===== Heatmap 2: Temperature vs Quality =====
print("Calculating T vs Q grid...")

# Create 2D grid
T_range = np.linspace(25, 350, N_grid)  # 25 to 350 degC (within two-phase region limits)
Q_range = np.linspace(0, 1, N_grid)  # Quality from 0 to 1
T_grid, Q_grid = np.meshgrid(T_range, Q_range)

# Flatten for calculation
T_flat = cast(Q[Temperature, Numpy1DArray], Q(T_grid.flatten(), "degC"))
Q_flat = cast(Q[Dimensionless, Numpy1DArray], Q(Q_grid.flatten(), "dimensionless"))

# Calculate properties for both formulations
w97_tq = Fluid("IF97::Water", T=T_flat, Q=Q_flat)
w95_tq = Fluid("Water", T=T_flat, Q=Q_flat)

# Create dataframe
df_tq = pl.DataFrame(
    {
        "T": T_flat.m,
        "Q": Q_flat.m,
    }
    | {f"w95_{a}": getattr(w95_tq, a).m for a in attrs}
    | {f"w97_{a}": getattr(w97_tq, a).m for a in attrs}
)

# Get statistics for dynamic color scaling
stats_tq = get_ratio_stats(df_tq, attrs)
print("\nRatio statistics (IAPWS-95 / IAPWS-97) for T vs Q:")
for a in attrs:
    if a in stats_tq:
        s = stats_tq[a]
        pct_diff = (s["mean"] - 1.0) * 100
        pct_range = (s["max"] - s["min"]) * 100
        print(
            f"  {a}: {s['min']:.6f} - {s['max']:.6f} (mean: {s['mean']:.6f}, "
            f"Δ={pct_diff:+.3f}%, range={pct_range:.3f}%)"
        )

# Create heatmaps with dynamic color limits
plots_tq = []
for a in attrs:
    if a not in stats_tq:
        continue

    # Calculate percentage difference instead of ratio for better visualization
    df_plot = df_tq.with_columns(((pl.col(f"w95_{a}") / pl.col(f"w97_{a}") - 1.0) * 100).alias(f"pct_diff_{a}"))

    # Use symmetric color limits around 0
    s = stats_tq[a]
    max_abs_pct = max(abs((s["min"] - 1.0) * 100), abs((s["max"] - 1.0) * 100))
    clim = (-max_abs_pct, max_abs_pct) if max_abs_pct > 0.001 else (-0.1, 0.1)

    plot = df_plot.hvplot.heatmap(
        x="T",
        y="Q",
        C=f"pct_diff_{a}",
        title=f"{a}: % Difference (95-97)/97 (T vs Q)",
        xlabel="Temperature (°C)",
        ylabel="Quality (vapor fraction)",
        cmap="RdBu_r",
        clim=clim,
        colorbar=True,
        width=500,
        height=400,
        clabel="% Difference",
    )
    plots_tq.append(plot)

print("\nT vs Q plots ready")
sum(plots_tq[1:], plots_tq[0]).cols(2)

Calculating T vs Q grid...

Ratio statistics (IAPWS-95 / IAPWS-97) for T vs Q:
  D: 0.999835 - 1.000423 (mean: 1.000100, Δ=+0.010%, range=0.059%)
  H: 0.999824 - 1.000193 (mean: 1.000003, Δ=+0.000%, range=0.037%)
  S: 0.999863 - 1.000177 (mean: 1.000003, Δ=+0.000%, range=0.031%)

T vs Q plots ready


:Layout
   .HeatMap.I   :HeatMap   [T,Q]   (pct_diff_D)
   .HeatMap.II  :HeatMap   [T,Q]   (pct_diff_H)
   .HeatMap.III :HeatMap   [T,Q]   (pct_diff_S)

In [8]:
# ===== Heatmap 3: Pressure vs Quality =====
print("Calculating P vs Q grid...")

# Create 2D grid
P_range = np.linspace(0.1, 200, N_grid)  # 0.1 to 200 bar
Q_range = np.linspace(0, 1, N_grid)  # Quality from 0 to 1
P_grid, Q_grid = np.meshgrid(P_range, Q_range)

# Flatten for calculation
P_flat = cast(Q[Pressure, Numpy1DArray], Q(P_grid.flatten(), "bar"))
Q_flat = cast(Q[Dimensionless, Numpy1DArray], Q(Q_grid.flatten(), "dimensionless"))

# Calculate properties for both formulations
w97_pq = Fluid("IF97::Water", P=P_flat, Q=Q_flat)
w95_pq = Fluid("Water", P=P_flat, Q=Q_flat)

# Create dataframe
df_pq = pl.DataFrame(
    {
        "P": P_flat.m,
        "Q": Q_flat.m,
    }
    | {f"w95_{a}": getattr(w95_pq, a).m for a in attrs}
    | {f"w97_{a}": getattr(w97_pq, a).m for a in attrs}
)

# Get statistics for dynamic color scaling
stats_pq = get_ratio_stats(df_pq, attrs)
print("\nRatio statistics (IAPWS-95 / IAPWS-97) for P vs Q:")
for a in attrs:
    if a in stats_pq:
        s = stats_pq[a]
        pct_diff = (s["mean"] - 1.0) * 100
        pct_range = (s["max"] - s["min"]) * 100
        print(
            f"  {a}: {s['min']:.6f} - {s['max']:.6f} (mean: {s['mean']:.6f}, "
            f"Δ={pct_diff:+.3f}%, range={pct_range:.3f}%)"
        )

# Create heatmaps with dynamic color limits
plots_pq = []
for a in attrs:
    if a not in stats_pq:
        continue

    # Calculate percentage difference instead of ratio for better visualization
    df_plot = df_pq.with_columns(((pl.col(f"w95_{a}") / pl.col(f"w97_{a}") - 1.0) * 100).alias(f"pct_diff_{a}"))

    # Use symmetric color limits around 0
    s = stats_pq[a]
    max_abs_pct = max(abs((s["min"] - 1.0) * 100), abs((s["max"] - 1.0) * 100))
    clim = (-max_abs_pct, max_abs_pct) if max_abs_pct > 0.001 else (-0.1, 0.1)

    plot = df_plot.hvplot.heatmap(
        x="P",
        y="Q",
        C=f"pct_diff_{a}",
        title=f"{a}: % Difference (95-97)/97 (P vs Q)",
        xlabel="Pressure (bar)",
        ylabel="Quality (vapor fraction)",
        cmap="RdBu_r",
        clim=clim,
        colorbar=True,
        width=500,
        height=400,
        clabel="% Difference",
    )
    plots_pq.append(plot)

print("\nP vs Q plots ready")
sum(plots_pq[1:], plots_pq[0]).cols(2)

Calculating P vs Q grid...

Ratio statistics (IAPWS-95 / IAPWS-97) for P vs Q:
  D: 0.998818 - 1.000366 (mean: 0.999986, Δ=-0.001%, range=0.155%)
  H: 0.999780 - 1.000397 (mean: 1.000032, Δ=+0.003%, range=0.062%)
  S: 0.999828 - 1.000312 (mean: 1.000029, Δ=+0.003%, range=0.048%)

P vs Q plots ready


:Layout
   .HeatMap.I   :HeatMap   [P,Q]   (pct_diff_D)
   .HeatMap.II  :HeatMap   [P,Q]   (pct_diff_H)
   .HeatMap.III :HeatMap   [P,Q]   (pct_diff_S)

## Speed Benchmarks

In [9]:
def benchmark_fluid(backend: str, n_points: int = 1000) -> dict[str, float]:
    """Benchmark fluid property calculations."""

    # Test case 1: Single-phase liquid (common in process engineering)
    T_test = cast(Q[Temperature, Numpy1DArray], np.linspace(Q(20, "degC"), Q(300, "degC"), n_points))
    P_test = Q(10, "bar")

    start = time.perf_counter()
    fluid = Fluid(backend, T=T_test, P=P_test)
    init_time = time.perf_counter() - start

    # Measure property access time
    start = time.perf_counter()
    _ = fluid.D
    _ = fluid.H
    _ = fluid.S
    _ = fluid.V
    _ = fluid.M
    property_time = time.perf_counter() - start

    total_time = init_time + property_time

    return {
        "init_time": init_time,
        "property_time": property_time,
        "total_time": total_time,
        "points": n_points,
    }


# Run benchmarks
print("Running benchmarks with 1000 points...")
results_97 = benchmark_fluid("IF97::Water", 1000)
results_95 = benchmark_fluid("Water", 1000)

print("\n" + "=" * 60)
print("BENCHMARK RESULTS")
print("=" * 60)
print("\nIAPWS-97:")
print(f"  Initialization: {results_97['init_time'] * 1000:.2f} ms")
print(f"  Property access: {results_97['property_time'] * 1000:.2f} ms")
print(f"  Total: {results_97['total_time'] * 1000:.2f} ms")

print("\nIAPWS-95:")
print(f"  Initialization: {results_95['init_time'] * 1000:.2f} ms")
print(f"  Property access: {results_95['property_time'] * 1000:.2f} ms")
print(f"  Total: {results_95['total_time'] * 1000:.2f} ms")

speedup = results_95["total_time"] / results_97["total_time"]
print(f"\nSpeedup: IAPWS-97 is {speedup:.1f}x faster than IAPWS-95")
print(f"Time saved per 1000 calculations: {(results_95['total_time'] - results_97['total_time']) * 1000:.2f} ms")
print("=" * 60)

Running benchmarks with 1000 points...

BENCHMARK RESULTS

IAPWS-97:
  Initialization: 0.01 ms
  Property access: 3.35 ms
  Total: 3.36 ms

IAPWS-95:
  Initialization: 0.00 ms
  Property access: 22.93 ms
  Total: 22.93 ms

Speedup: IAPWS-97 is 6.8x faster than IAPWS-95
Time saved per 1000 calculations: 19.57 ms


In [10]:
# Test two-phase calculations (common in steam systems)
print("Testing two-phase (saturated steam) calculations...")

Q_test = cast(Q[Dimensionless, Numpy1DArray], np.linspace(Q(0), Q(1), 1000))
P_sat = Q(10, "bar")

start = time.perf_counter()
fluid_97_sat = Fluid("IF97::Water", Q=Q_test, P=P_sat)
_ = fluid_97_sat.D, fluid_97_sat.H, fluid_97_sat.S
time_97_sat = time.perf_counter() - start

start = time.perf_counter()
fluid_95_sat = Fluid("Water", Q=Q_test, P=P_sat)
_ = fluid_95_sat.D, fluid_95_sat.H, fluid_95_sat.S
time_95_sat = time.perf_counter() - start

print("\nTwo-phase calculations (1000 points):")
print(f"  IAPWS-97: {time_97_sat * 1000:.2f} ms")
print(f"  IAPWS-95: {time_95_sat * 1000:.2f} ms")
print(f"  Speedup: {time_95_sat / time_97_sat:.1f}x")

Testing two-phase (saturated steam) calculations...

Two-phase calculations (1000 points):
  IAPWS-97: 2.69 ms
  IAPWS-95: 5.99 ms
  Speedup: 2.2x
